## Design your own trigger

The trigger decides what type of physics we record forever. While ATLAS has a suite of inclusive triggers, for example a trigger for events containing at least one electron/positron above 24 GeV or at least one hadronic jet above 400 GeV, there is also room for more specialised triggers that looks for events that look like a specific Standard Model or hypothetical new process. 
Good specialiased triggers do two things:
- They have a good signal-to-noise ratio.
- They select events at a low/affordable rate.

Today, we are interested in the following process, the production of a Higgs boson via Vector Boson Fusion, which then decays to two photons, as shown below.


The current single photon trigger selects events containing a photon of at least 200 GeV. Can we use the event topology to design a new trigger that makes our event selection more rich with events containing VBH Higgs$\rightarrow \gamma + \gamma$?

<img img align="left" src="https://www.physicsforums.com/attachments/vbf-png.205986/" width="400" />
<img img align="left" src="https://i.stack.imgur.com/RCO4y.png" width="400" />

## Trigger limitations

Our trigger will have to fit into the current ATLAS rate budget. We have room for an additional trigger with a rate of 2 Hz. The current 300 GeV single photon trigger has a rate of 30 Hz and a significance ($S/\sqrt{B}$) of 0.9 for the VBF $H\rightarrow \gamma\gamma$ process. Can we improve that?

| 300 GeV photon trigger Rate | VBF H trigger Max Rate | Rel. Reduction |
| --- | --- | --- |
| 30 Hz | 5 Hz | 0.16



### How to design your trigger?

The code below sets up some histograms that hint at useful event features that show a marked difference between signal (VBF H$\rightarrow \gamma\gamma$) and background (data). You can use these features to apply an event selection that will enhance the signal-to-background ratio.

**Remember**: Additional selections means we can lower the high 300 GeV threshold on the leading photon!

You can also have a look at the following paper to get inspiration:
["Search for invisible Higgs-boson decays in events with vector-boson fusion signatures using 139 fb−1 of proton-proton data recorded by the ATLAS experiment"](https://arxiv.org/abs/2202.07953)


### The final goal?

Design a trigger that is affordable and has a high significance! A significance of 3 is good, of 5 is very good!

In [ ]:
import ROOT

# -- set up of common variables ---

tree_name = "mini" # event "tree" in which information of each event in the data set are defined:
                   # event level information, particles and their properties

GeV = 1e-3 # convert MeV to GeV        

In [ ]:
# --- list of histograms --- 
# NOTE: Not all histograms are filled in the analysis code. 
# They hint at what could be interesting to look at, though!

bin_width = 5 # GeV
h_pt_min = 0
h_pt_max = 450
pt_bins = int((h_pt_max-h_pt_min)/bin_width)

def get_histograms():
    # photon histograms
    hist_list = {}
    hist_list["photon0_pt"] = ROOT.TH1F("h_photon0_pt","leading photon pT; offline photon pT [GeV]; events",pt_bins,h_pt_min,h_pt_max)
    hist_list["photon1_pt"] = ROOT.TH1F("h_photon1_pt","subleading photon pT; offline photon pT [GeV]; events",pt_bins,h_pt_min,h_pt_max)

    hist_list["jet_n"] = ROOT.TH1F("h_jet_n","jet multiplicity; jet multiplicity; events", 10, 0, 10)
    
    hist_list["jet0_pt"]= ROOT.TH1F("h_jet0_pt","leading jet pT; offline jet pT [GeV]; events", pt_bins,h_pt_min,h_pt_max)
    hist_list["jet1_pt"] = ROOT.TH1F("h_jet1_pt","subleading jet pT; offline jet pT [GeV]; events", pt_bins,h_pt_min,h_pt_max)

    hist_list["jet0_eta"] = ROOT.TH1F("h_jet0_eta","leading jet eta; offline jet eta; events",100,-5,5)
    hist_list["jet1_eta"] = ROOT.TH1F("h_jet1_eta","subleading jet eta; offline jet eta; events",100,-5,5)
    hist_list["dijet_mass"] = ROOT.TH1F("h_dijet_mass","dijet mass; dijet mass [GeV]; events", pt_bins,h_pt_min,h_pt_max)
    
    hist_list["dijet_dphi"] = ROOT.TH1F("h_dijet_dphi","dijet delta phi; dijet dphi; events", 340, 0, 3.4)

    hist_list["dijet_deta"] = ROOT.TH1F("h_dijet_deta","dijet delta eta; dijet deta; events", 500, 0, 5)
    
    hist_list["cutflow"] = ROOT.TH1I("cutflow", "cutflow;cut;count", 10,0,10)
    
    for hname, hist in hist_list.items():
        hist.SetDirectory(0)

    
    return hist_list

In [ ]:
# ------------------------------
# --- Trigger Selection Code --- 
# ------------------------------
# This is where we implement the event selection of the trigger based on objects and combined object systems.

def analyse_events(tree, hist_list):
    
    for hname, hist in hist_list.items():
        hist.Reset()

    for event in tree:
    
        hist_list["cutflow"].Fill("total",1)

        if event.photon_n < 1: continue
        hist_list["cutflow"].Fill("1Photon",1)
        leading_photon_pt = event.photon_pt[0] * GeV   
        hist_list["photon0_pt"].Fill(leading_photon_pt) 
        if leading_photon_pt < 300: continue
        hist_list["cutflow"].Fill("300GeVPhoton",1)

        # Let's first reconstruct leading photons...
        if event.photon_n < 2: continue
        photon0 = ROOT.TLorentzVector()
        photon0.SetPtEtaPhiE(event.photon_pt[0], event.photon_eta[0], event.photon_phi[0], event.photon_E[0])                            
        photon1 = ROOT.TLorentzVector()
        photon1.SetPtEtaPhiE(event.photon_pt[1], event.photon_eta[1], event.photon_phi[1], event.photon_E[1])
        subleading_photon_pt = photon1.Pt() * GeV
        hist_list["photon1_pt"].Fill(subleading_photon_pt)
              
        # The first 2 jets that don't overlap with photons might be interesting for our selection
        jets = []
        for jidx in range(event.jet_n):
            jet = ROOT.TLorentzVector()
            jet.SetPtEtaPhiE(event.jet_pt[jidx], event.jet_eta[jidx], event.jet_phi[jidx], event.jet_E[jidx])
            if photon0.DeltaR(jet) > 0.4 and photon1.DeltaR(jet) > 0.4:
                jets.append(jet)
        
        hist_list["jet_n"].Fill(len(jets))
        
        #if len(jets) < 2: continue
        
        # <FILL ME>

        

In [ ]:
# ------------------------------
# --- ANALYSE MC SIGNAL ------
# ------------------------------
### You should not have to change anything here.

signal_hist_list = get_histograms()

base_url = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/MC"
input_name = "mc_345041.VBFH125_gamgam.GamGam.root"

infile = ROOT.TFile.Open(f"{base_url}/{input_name}")
tree = infile.Get("mini")
total_evts = tree.GetEntries()

for event in tree:
     xsection = event.XSection
     print("Signal x-section = %.4f fb"%(xsection))
     break

print("Analysing %.2f events"%(total_evts*xsection))        
        
analyse_events(tree, signal_hist_list)

In [ ]:
# ------------------------------
# --- ANALYSE BACKGROUND DATA ------
# ------------------------------
### You should not have to change anything here.

bg_hist_list = get_histograms()

base_url = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data"
input_name = "data_B.GamGam.root"

infile = ROOT.TFile.Open(f"{base_url}/{input_name}")
infile.GetListOfKeys().Print()
tree = infile.Get("mini")
total_evts = tree.GetEntries()

print(f"Analysing {total_evts} events")
analyse_events(tree, bg_hist_list)

In [ ]:
# ---------------------
# -- Plotting Code ---
# ---------------------
# Here we have a look at the distributions. Which distributions look very different
# for signal (red) and background (blue) events? Can we use those differences to enhance
# the signal selection of our trigger?
# (NOTE: Ignore errors about histogram normalisation. This will happen for histograms we have not filled.)


canvas = ROOT.TCanvas("Canvas", "c", 800, 600)
canvas.Divide(3,4,0.001,0.001)

cnv = 1
for hname in signal_hist_list:
    signal_hist = signal_hist_list[hname]
    bg_hist = bg_hist_list[hname]
    signal_hist.SetFillColorAlpha(ROOT.kRed, 0.5)
    bg_hist.SetFillColorAlpha(ROOT.kBlue, 0.5)
    canvas.cd(cnv)
    if not "cutflow" in hname:
        bg_hist.DrawNormalized("HIST")
        signal_hist.DrawNormalized("HISTSAME")
    canvas.Draw()
    cnv+=1

In [ ]:
canvas2 = ROOT.TCanvas("Canvas2", "c2", 800, 600)

signal_hist_cp = signal_hist_list["cutflow"]
signal_hist_cp.Draw("H")
bg_hist_cp = bg_hist_list["cutflow"]
bg_hist_cp.Draw("HSAME")
canvas2.Draw()

In [ ]:
# ----------------------
# --- Final Results ----
# ----------------------
# Now we calculate the significance of our signal after selection and our trigger "rate"
# NOTE: Signal events were multiplied by their cross section during the event analysis already
# while background events are scaled by a factor 5 here as we have only analysed 1/5 of the 10 ifb dataset.

import math

signal_events = signal_hist_list["cutflow"].GetMinimum() * xsection
bg_events = bg_hist_list["cutflow"].GetMinimum() * 5

reduction = bg_events/19635 # denominator: Number of events of 300 GeV photon trigger
lumi_factor = 30. # 30 * 10 inverse fb = 300 ifb, the total amount collected by end of Run 3.

print("Number of passed signal events = ", signal_events)
print("Number of passed bg events = ", bg_events)
print("Significance = %.2f "%(signal_events*lumi_factor/math.sqrt(bg_events*lumi_factor)))
print("Background rate relative to 300 GeV photon trigger = ", reduction )
if reduction > 0.16:
    print(" -- Sorry, we can't afford this trigger.")